#  (Limpieza) y Transformación de Datos
<img src='imgs/fancy_process.png' alt="Smiley face" width="700"><br>

### Se puede facilitar el proceso de aprendizaje por medio de seleccion, creación o reducción de atributos
* SelectKBest
* PCA
* One-Hot Encoder

Recordemos los atributos disponibles en el dataset de iris:
![Iris with labels](imgs/iris_with_labels.jpg)

In [ ]:
# Imports for python 2/3 compatibility

from __future__ import absolute_import, division, print_function, unicode_literals

# For python 2, comment these out:
# from builtins import range

### Seleccionar los mejores K atributos (también reducción de dimensiones)
* Esto se considera aprendizaje no supervisado

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# SelectKBest para seleccionar los mejores atributos

from sklearn import datasets
from sklearn.feature_selection import SelectKBest, chi2

# Nuestros datos limpios (no siempre va a ser tan fácil)
iris = datasets.load_iris()
X, y = iris.data, iris.target

print('Tamaño Original:', X.shape)

In [ ]:
# Agregar un nuevo atributo: La proporción entre las medidas del iris
df = pd.DataFrame(X, columns = iris.feature_names)
df['petal width / sepal width'] = df['petal width (cm)'] / df['sepal width (cm)']
new_feature_names = df.columns
print('New feature names:', list(new_feature_names))

# Hemos agregado un nuevo atributo a los datos
X = np.array(df)

In [ ]:
# Seleccion de Atributos:
#  la entrada es una función de scoring (acá chi2) que obtienen valores P univariate
#  y el número de los K atributos con mejor score, acá elegimos 3
dim_red = SelectKBest(chi2, k = 3)
dim_red.fit(X, y)
X_t = dim_red.transform(X)

In [ ]:
# Mostramos los scores, atributos seleccionados y nuevo shape
print('Scores:', dim_red.scores_)
print('New shape:', X_t.shape)

In [ ]:
# Obtener columnas seleccionadas
selected = dim_red.get_support() # Valores booleanos
selected_names = new_feature_names[selected]

print('Mejores k atributos: ', list(selected_names))

**Nota acerca de la función de scoring en transformaciones con  `SelectKBest`:**
* Para regresión - [f_regression](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)
* Para classificación - [chi2](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2), [f_classif](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif)


### Análisis de Componente Principal (aka PCA)
* Reduce dimensiones (número de atributos), basándose en que información explica la mayor varianza.
* Se considera aprendizaje no supervisado
* Es útil cuando se tiene un espacio de atributos muy grande (por ejemplo, si el encargado del dataset hubiese hecho 100 medidas de las flores)
* Más acerca de PCA en Wikipedia [acá](https://en.wikipedia.org/wiki/Principal_component_analysis)

In [ ]:
# PCA para reducción de dimensiones

from sklearn import decomposition
from sklearn import datasets

iris = datasets.load_iris()

X, y = iris.data, iris.target

# Ejecutar principal component analysis
pca = decomposition.PCA(.95)
pca.fit(X)
X_t = pca.transform(X)
(X_t[:, 0])

# Importar numpy and matplotlib para visualización
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
%matplotlib inline

# Separamos los datos basándonos en los primeros dos componentes principales
x1, x2 = X_t[:, 0], X_t[:, 1]


#  (nota: los nombres de iris pueden conseguirse de iris.target_names, también en docs)
c1 = np.array(list('rbg')) # colors
colors = c1[y] # y definido por color
classes = iris.target_names[y] # y definido por nombre de iris
for (i, cla) in enumerate(set(classes)):
    xc = [p for (j, p) in enumerate(x1) if classes[j] == cla]
    yc = [p for (j, p) in enumerate(x2) if classes[j] == cla]
    cols = [c for (j, c) in enumerate(colors) if classes[j] == cla]
    plt.scatter(xc, yc, c = cols, label = cla)
    plt.ylabel('Principal Component 2')
    plt.xlabel('Principal Component 1')
plt.legend(loc = 4)

### Más métodos de selección de atributos [acá](http://scikit-learn.org/stable/modules/feature_selection.html)

### One Hot Encoding
* It's an operation on feature labels - a method of dummying variable
* Expands the feature space by nature of transform - later this can be processed further with a dimensionality reduction (the dummied variables are now their own features)
* FYI:  One hot encoding variables is needed for python ML module `tenorflow`
* Can do this with `pandas` method or a `sklearn` one-hot-encoder system

#### `pandas` method

In [ ]:
# Dummy variables with pandas built-in function

import pandas as pd
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data, iris.target

# Convert to dataframe and add a column with actual iris species name
data = pd.DataFrame(X, columns = iris.feature_names)
data['target_name'] = iris.target_names[y]

df = pd.get_dummies(data, prefix = ['target_name'])
df.head()

#### `sklearn` method

In [ ]:
# OneHotEncoder for dummying variables

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data, iris.target

# We encode both our categorical variable and it's labels
enc = OneHotEncoder()
label_enc = LabelEncoder() # remember the labels here

# Encode labels (can use for discrete numerical values as well)
data_label_encoded = label_enc.fit_transform(y)

# Encode and "dummy" variables
data_feature_one_hot_encoded = enc.fit_transform(y.reshape(-1, 1))
print(data_feature_one_hot_encoded.shape)

num_dummies = data_feature_one_hot_encoded.shape[1]
df = pd.DataFrame(data_feature_one_hot_encoded.toarray(), columns = label_enc.inverse_transform(range(num_dummies)))

df.head()

Created by a Microsoft Employee.
	
The MIT License (MIT)<br>
Copyright (c) 2016 Micheleen Harris